<a href="https://colab.research.google.com/github/datavzch/GANWS21/blob/main/DCGAN_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Required Libraries**


In [ ]:
# Load required Libraries
%matplotlib inline
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import cv2
import os


# **Define Discriminator and Generator**

In [ ]:
# The discriminator model takes a sample image (from our dataset) as input and outputs a binary classification prediction.
# Input: 32 x 32 pixels 3 color channel (rgb) image [32,32,3]
# Output: Binary classification (0.0 to 1.0 - is it real or fake).

def define_discriminator(input_shape=(32,32,3)):
    model = keras.models.Sequential()
    # The discriminator is composed of a set convolutional layers, 
    # 1 normal convolutional layer followed by a set of 
    # 3 convolutional layers with a 2 x 2 stride. Together this layers 
    # Downsample the image from 32 x 32 x 3 all the way to 4 x 4 x 3.
    # Input is a 32*32*3 image
    model.add(keras.layers.Conv2D(filters=64,kernel_size= (3,3),padding = 'same',input_shape = input_shape))
    
    # The use of LeakyReLU, dropout and Adam as well as their 
    # parameters are derived from the documentation and good practices suggestions. 

    model.add(keras.layers.LeakyReLU(0.2))
    #model.add(keras.layers.Dropout(0.4))

    # Downsample 16*16*3 image
    model.add(keras.layers.Conv2D(filters= 128,kernel_size= (3,3),strides = (2,2),padding = 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    #model.add(keras.layers.Dropout(0.4))
    
    # Downsample 8*8*3 image
    model.add(keras.layers.Conv2D(filters= 128,kernel_size= (4,4),strides = (2,2),padding = 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    #model.add(keras.layers.Dropout(0.4))
    
    # Downsample 4*4*3
    model.add(keras.layers.Conv2D(filters= 256, kernel_size= (4,4), strides = (2,2), padding = 'same'))
    model.add(keras.layers.LeakyReLU(0.2))
    #model.add(keras.layers.Dropout(0.4))
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.4))

    # We have a single node in the output layer with 
    # sigmoid activation function for the prediction (real or fake)

    model.add(keras.layers.Dense(units= 1,activation = 'sigmoid'))
    opt = keras.optimizers.Adam(learning_rate = 0.0002, beta_1 = 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt, metrics = ['accuracy'])
    
    return model

In [ ]:
# The generator should output new fake images.
# It takes a point in the latent space as input and outputs a 32 x 32 rgb image.
# By assigning meaning to this latent points during the training process the latent space turns into a compressed repressentation of the dataset, generating "real" images.

# Input: Point in latent space
# Output: Rgb Image of 32 × 32 pixels (pixel values between -1 - 1)

def define_generator(latent_dim):
  # use a hidden dense layer to represent a low resolution version of the output (4 x 4 x 3).
  # Give enough nodes (256 - Arbitrary - can be changed) for multiple versions 
  # of the output image (more feature maps).
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units= 256 * 4 * 4, input_dim = latent_dim)) #4 * 4 Arbitrary - Can be changed to test different results
    model.add(keras.layers.LeakyReLU(0.2)) # LeakyReLU with a default slope of 0.2, reported as a good practice. 
    model.add(keras.layers.Reshape((4, 4, 256)))
    # Pass the resulting feature maps into convolutional layer. 
    # Upsample the image to higher resolution.
    # Use Conv2DTranspose for upsampling with a stride of 2x2 
    # Repeat upsampling until achieving our 32 x 32 size goal.

    # Upsample 8 * 8 
    model.add(keras.layers.Conv2DTranspose(filters= 128, kernel_size = (4,4),padding='same', strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # Upsample 16 * 16 
    model.add(keras.layers.Conv2DTranspose(filters= 128, kernel_size = (4,4),padding='same', strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))
    # Upsample  32 * 32 now
    model.add(keras.layers.Conv2DTranspose(filters= 128, kernel_size = (4,4),padding='same', strides= (2,2)))
    model.add(keras.layers.LeakyReLU(0.2))

    # Conv2D output layer with 3 filters (3 channels) with a kernel size of 3 
    # for generation of single feature map with preserved dimensions (32 x 32 x 3)
    # Use Than activation (good practice) to ensure the values are in range (from -1 to 1).
    model.add(keras.layers.Conv2D(filters= 3, kernel_size= (3,3),padding = 'same', activation= 'tanh')) 

    # the generated images are completely random pixel values in [-1, 1], rescaled to [0, 1].
    
    return model

# **Define GAN**

In [ ]:
# Generator weights are updated based in the discriminator performance. 
# The Gan function is defined to send the output from the generator to the discriminator, classified it, and then update generators weight. 
# Mark generator samples as real samples (trick discriminator)
# Discriminator is marked as not trainable (to avoid over trainning with fake samples).
def define_GAN(g_model, d_model):
    d_model.trainable = False
    model = keras.models.Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = keras.optimizers.Adam(learning_rate= 0.0002,beta_1= 0.5)
    model.compile(loss= 'binary_crossentropy', optimizer= opt)
    return model

# **Generate Sample**

In [ ]:
# Canadian Institute For Advanced Research dataset of 60,000 32 × 32 pixel color photographs of objects from 10 classes.
# https://www.cs.toronto.edu/~kriz/cifar.html
def load_data():
    (X_train, _), (_, _) = keras.datasets.cifar10.load_data()
    return X_train


In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    X = np.random.randn(latent_dim * n_samples) # generate points in the latent space
    X = X.reshape((n_samples, latent_dim)) # reshape into a batch of inputs for the network
    return X

In [ ]:
# select real samples
def generate_real_sample(n_samples):
    data = load_data() #we are loading the data withouth labels
    ix = np.random.randint(0,data.shape[0], n_samples)
    X = data[ix] # retrieve selected random images
    X = X.reshape((n_samples, 32, 32, 3)).astype('float32')
    X = (X - 127.5) / 127.5 # scale from [0,255] to [-1,1]
    y = np.ones((n_samples, 1)) # create real class labels
    return X, y

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_sample(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim= latent_dim,n_samples= n_samples)  # generate points in latent space
    X = g_model.predict(x_input) # predict outputs
    y = np.zeros((n_samples, 1)) # create class labels 
    return X, y

# **Summarize and Plot**

In [ ]:
# create and save a plot of generated images
def save_plot(x_input, epoch, n=7):
    x_input = (x_input + 1.0) / 2.0
    filename = f'generated_{epoch + 1}.png'
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.imshow(x_input[i,:,:,:])
        plt.axis('off')
    plt.savefig(filename)
    plt.show()
    plt.close()

In [ ]:
# Evaluate discriminator, plot generated images, save generator model
def summarize_the_model(g_model, d_model, epoch, latent_dim, n_samples):
    X_real, y_real = generate_real_sample(n_samples= n_samples)
    X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                          latent_dim= latent_dim,
                                          n_samples= n_samples)
    print(f'Accuracy on real data: {d_model.evaluate(X_real, y_real, verbose= 0)}')
    print(f'Accuracy on fake data: {d_model.evaluate(X_fake, y_fake, verbose= 0)}')
    filename = f'model_e_{epoch + 1}.h5'
    save_plot(x_input= X_fake,
              epoch= epoch)
    
    g_model.save(filename)

# **Model Training**

In [ ]:
# train the generator and discriminator
def train_GAN(gan_model, g_model, d_model, dataset_len, latent_dim, iters= 200, batch_size= 256):
    half_batch = int(batch_size / 2)
    batch_per_epoch = int(dataset_len / batch_size)
    for i in range(iters):
        for j in range(batch_per_epoch):
            X_real, y_real = generate_real_sample(n_samples= half_batch)
            X_fake, y_fake = generate_fake_sample(g_model= g_model,
                                                  latent_dim= latent_dim,
                                                  n_samples= half_batch)
            X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
            d_model.train_on_batch(X, y)
            
            x_gan = generate_latent_points(latent_dim= latent_dim,
                                             n_samples= batch_size)
            y_gan = np.ones((batch_size, 1))
            gan_model.train_on_batch(x_gan, y_gan)
            if not j%100: 
                print(f'Epoch: {i+1}, Batches/Epoch: {j+1}/{batch_per_epoch}')
                summarize_the_model(g_model= g_model,
                                    d_model= d_model,
                                    epoch= i,
                                    latent_dim= latent_dim,
                                    n_samples= batch_size)

In [ ]:
# size of the latent space
latent_dim = 100
g_model = define_generator(latent_dim= latent_dim) 
d_model = define_discriminator()
gan_model = define_GAN(d_model= d_model,
                       g_model= g_model)
train_GAN(gan_model= gan_model, 
          g_model= g_model,
          d_model= d_model,
          dataset_len= load_data().shape[0],
          latent_dim= latent_dim)

# **Prediction on resulting model**

In [ ]:
# Load pretrained model 
model = keras.models.load_model('./model_e_99.h5', compile= False)

In [ ]:
# Plot pre trained model prediction
vec = np.array([[np.cos(_) for _ in range(100)]])
X = ( model.predict(vec) + 1 ) / 2.0
plt.imshow(X[0])
plt.show()

In [ ]:
# Define plotting for 49 image sample
def plotModel(model, n_samples=49):
    plt.figure(figsize=(18,9))
    for i in range(n_samples):
        ran = np.random.randn(n_samples, 100)
        sqrt = int(np.sqrt(n_samples))
        plt.subplot(sqrt, sqrt, i+1)
        X = model.predict(ran)
        X = (X + 1) / 2.0
        plt.imshow(X[0])
        plt.axis('off')
    plt.show()

In [ ]:
plotModel(model, n_samples= 25)